In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [ ]:
df_epochs = pd.read_csv('csv/epochs.csv')
df_rec = pd.read_csv('csv/recalls.csv')
df_prec = pd.read_csv('csv/precisions.csv')

In [ ]:
# remove columns that contain MIN or MAX
df_rec = df_rec.loc[:,~df_rec.columns.str.contains('MIN|MAX')]
df_prec = df_prec.loc[:,~df_prec.columns.str.contains('MIN|MAX')]
df_epochs = df_epochs.loc[:,~df_epochs.columns.str.contains('MIN|MAX')]

In [ ]:
df_rec

In [ ]:
df_epochs.columns = [k if k=='Step' else k.split('_')[2] for k in df_epochs.columns]
df_rec.columns = [k if k=='Step' else k.split('_')[2] for k in df_rec.columns]
df_prec.columns = [k if k=='Step' else k.split('_')[2] for k in df_prec.columns]

In [ ]:
# combine all columns df epochs into one
df_epochs = df_epochs.bfill(axis=1).iloc[:,:2]
df_epochs.columns = ['Step', 'epoch']

In [ ]:
df_rec['epoch'] = df_epochs['epoch']
df_prec['epoch'] = df_epochs['epoch']

In [ ]:
def f1_score(recall, precision):
    return 2 * (precision * recall) / (precision + recall)

In [ ]:
rec_mean = (df_rec.groupby('epoch').mean().groupby(level=0, axis=1).mean() * 100)
rec_std = (df_rec.groupby('epoch').mean().groupby(level=0, axis=1).std() * 100)
rec_mean.round(2)

In [ ]:
rec_std.round(2)

In [ ]:
prec_mean = (df_prec.groupby('epoch').mean().groupby(level=0, axis=1).mean() * 100)
prec_std = (df_prec.groupby('epoch').mean().groupby(level=0, axis=1).std() * 100)
prec_mean.round(2)

In [ ]:
prec_std.round(2)

In [ ]:
f1_mean = f1_score(df_rec.groupby('epoch').mean(), df_prec.groupby('epoch').mean()).groupby(level=0, axis=1).mean() * 100
f1_std = f1_score(df_rec.groupby('epoch').mean(), df_prec.groupby('epoch').mean()).groupby(level=0, axis=1).std() * 100
f1_mean.round(2)

In [ ]:
f1_std.round(2)

In [ ]:
models = ['glofnd', 'fnd', ]
new_names = ['GloFND','FNC', ]

In [ ]:
prec_mean = prec_mean[models]
rec_mean = rec_mean[models]
prec_std = prec_std[models]
rec_std = rec_std[models]
f1_mean = f1_mean[models]
f1_std = f1_std[models]

prec_mean.columns = new_names
rec_mean.columns = new_names
prec_std.columns = new_names
rec_std.columns = new_names
f1_mean.columns = new_names
f1_std.columns = new_names

In [ ]:
label_font = 40
tick_font = 35
legend_font = 30
linewidth = 6

In [ ]:
plt.figure(figsize=(9, 6), dpi=300)

for m in new_names:
        ax = prec_mean[m].plot(ax=plt.gca(), linewidth=linewidth, marker=None)
        # ax.fill_between(prec_mean.index, 
        #         prec_mean[m] - prec_std[m] * 5, 
        #         prec_mean[m] + prec_std[m] * 5, 
        #         alpha=1)  # Shaded std region

plt.xlabel('Epoch', fontsize=label_font)
plt.ylabel('Precision (%)', fontsize=label_font)
plt.xticks(fontsize=32)
plt.yticks(fontsize=tick_font)

# plt.grid(which='major', axis='y')
plt.tight_layout()
plt.legend(fontsize=legend_font)

plt.axvline(x=83, alpha=0.4, color='r', linestyle='--')
plt.axvline(x=87, alpha=0.4, color='r', linestyle='--')
ax.axvspan(83, 87, color='red', alpha=0.4)  # Red fill with alpha 0.3

plt.savefig(f"plots/precision.pdf", format='pdf')

In [ ]:
plt.figure(figsize=(9, 6), dpi=300)

for m in new_names:
        ax = rec_mean[m].plot(ax=plt.gca(), linewidth=linewidth, marker=None)
        # ax.fill_between(prec_mean.index, 
        #         prec_mean[m] - prec_std[m] * 5, 
        #         prec_mean[m] + prec_std[m] * 5, 
        #         alpha=1)  # Shaded std region

plt.xlabel('Epoch', fontsize=label_font)
plt.ylabel('Recall (%)', fontsize=label_font)
plt.xticks(fontsize=32)
plt.yticks(fontsize=tick_font)

plt.axvline(x=83, alpha=0.4, color='r', linestyle='--')
plt.axvline(x=87, alpha=0.4, color='r', linestyle='--')
ax.axvspan(83, 87, color='red', alpha=0.4)  # Red fill with alpha 0.3

# plt.grid(which='major', axis='y')
plt.tight_layout()
plt.legend(fontsize=legend_font)

plt.savefig(f"plots/recall.pdf", format="pdf")

In [ ]:
from sklearn.metrics import auc
plt.figure(figsize=(9, 6), dpi=300)

for m in new_names:
        ax = f1_mean[m].plot(ax=plt.gca(), linewidth=linewidth, marker=None)
        # ax.fill_between(prec_mean.index, 
        #         prec_mean[m] - prec_std[m] * 5, 
        #         prec_mean[m] + prec_std[m] * 5, 
        #         alpha=1)  # Shaded std region

plt.xlabel('Epoch', fontsize=label_font)
plt.ylabel('F1-Score (%)', fontsize=label_font)
plt.xticks(fontsize=32)
plt.yticks(fontsize=tick_font)

plt.axvline(x=83, alpha=0.4, color='r', linestyle='--')
plt.axvline(x=87, alpha=0.4, color='r', linestyle='--')
ax.axvspan(83, 87, color='red', alpha=0.4)  # Red fill with alpha 0.3

# plt.grid(which='major', axis='y')
plt.tight_layout()
plt.legend(fontsize=legend_font)

plt.savefig(f"plots/f1.pdf", format="pdf")

In [ ]:
(f1_mean['FNC'] > f1_mean['GloFND']).to_numpy()

In [ ]:
(f1_mean['GloFND'] - f1_mean['FNC']).mean()

In [ ]:
x = f1_mean.loc[120:].to_numpy()
x[:,1] *= -1
diff = x.sum(1)
diff

In [ ]:
print(f"Max {diff.max()}, min {diff.min()}, mean {diff.mean()}")